# Evaluate Performance for Flowrate Problem

In [23]:
from os import path
import rareeventestimation as ree
import pandas as pd
import plotly.express as px
from rareeventestimation.evaluation.constants import INDICATOR_APPROX_LATEX_NAME, BM_SOLVER_SCATTER_STYLE, MY_LAYOUT, DF_COLUMNS_TO_LATEX, LATEX_TO_HTML, WRITE_SCALE, DOUBLE_PRECISION
import plotly.graph_objects as go
from IPython.display import display, Markdown
# recommended: use autoreload for development: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data
### Option 1: Get precomputed data online

In [24]:
# data is here: https://archive.org/details/konstantinalthaus-rareeventestimation-data
# you can got to this link and inspect the files pefore loading
df_bm_agg = pd.read_json("https://ia801504.us.archive.org/23/items/konstantinalthaus-rareeventestimation-data/oscillator_problem_benchmark_aggregate.json")
df_bm = pd.read_json("https://ia801504.us.archive.org/23/items/konstantinalthaus-rareeventestimation-data/oscillator_problem_benchmark_processed.json")
df_agg = pd.read_json("https://ia801504.us.archive.org/23/items/konstantinalthaus-rareeventestimation-data/cbree_oscillator_problem_aggregated.json")
df = pd.read_json("https://ia601504.us.archive.org/23/items/konstantinalthaus-rareeventestimation-data/cbree_oscillator_problem_processed.json")

### Option 2: Aggregate locally precomputed data

In [25]:
# # uncomment to load existing data 
# # or to compile data after computing it yourself:

# data_dir ="/Users/konstantinalthaus/Documents/Master-TUM/Masterthesis/data/cbree_sim/nonlinear_oscillator"

# path_df = path.join(data_dir, "cbree_oscillator_problem_processed.json")
# path_df_agg = path.join(data_dir, "cbree_oscilltor_problem_aggregated.json")
# if not (path.exists(path_df) and path.exists(path_df_agg)):
#     df = ree.load_data(data_dir, "*")
#     df.drop(columns=["index", "Unnamed: 0",  "VAR Weighted Average Estimate","CVAR", "callback"], inplace=True)
#     df.drop_duplicates(inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     # Round parameters to compare floats safely
#     for col in [c for c in df.columns if c in DF_COLUMNS_TO_LATEX.keys()]:
#         if isinstance(df[col].values[0], float):
#             df[col] = df[col].round(5)
#     # melt aggregated estimates
#     df = df.rename(columns={"Estimate": "Last Estimate"})\
#         .melt(id_vars = [c for c in df.columns if not "Estimate" in c],
#               var_name="Averaging Method",
#               value_name="Estimate")
#     df = df.apply(ree.expand_cbree_name, axis=1, columns = ["Averaging Method", "observation_window"])
#     # pretty names
#     df = df.rename(columns=DF_COLUMNS_TO_LATEX)
#     #process data: add evaluations etc
#     df = ree.add_evaluations(df)
#     df_agg = ree.aggregate_df(df)
#     #save
#     df.to_json(path_df, double_precision = DOUBLE_PRECISION)
#     df_agg.to_json(path_df_agg, double_precision=DOUBLE_PRECISION)
# else:
#     df = pd.read_json(path_df)
#     df_agg = pd.read_json(path_df_agg)
# # load benchmarks
# data_dirs_bm = {
#     "enkf": "/Users/konstantinalthaus/Documents/Master-TUM/Masterthesis/data/enkf_sim_oscillator",
#     "sis": "/Users/konstantinalthaus/Documents/Master-TUM/Masterthesis/data/sis_sim_oscillator"
# }
# df_names_bm = {
#     "df": "oscillator_problem_benchmark_processed.json",
#     "df_agg": "oscillator_problem_benchmark_aggregate.json"
# }
# df_bm, df_bm_agg = ree.get_benchmark_df(data_dirs=data_dirs_bm,
#                                         df_names=df_names_bm,
#                                         df_dir="/Users/konstantinalthaus/Documents/Master-TUM/Masterthesis/rareeventestimation/docs/benchmarking/data",
#                                         force_reload=True,
#                                         remove_outliers=False)

## Compare relative Efficiency

In [26]:
# filter
my_mixture_model = "GM"
my_obs_windows = 2
my_epsilon = 1
my_bm_cvar_tgt = 1
prob = df_agg.Problem.unique().item() 
this_df_agg = df_agg.query(
    "Problem == @prob & `Averaging Method`=='Average Estimate' & mixture_model==@my_mixture_model")
this_df_agg = this_df_agg[this_df_agg['$N_{{ \\text{{obs}} }}$']
                          == my_obs_windows]
this_df_agg = this_df_agg[this_df_agg['$\\epsilon_{{\\text{{Target}}}}$'] == my_epsilon]
this_df_agg = this_df_agg[this_df_agg['$\\Delta_{{\\text{{Target}}}}$'] == my_bm_cvar_tgt] 
this_df = df.query("Problem == @prob & `Averaging Method`=='Average Estimate' & mixture_model==@my_mixture_model")
this_df = this_df[this_df['$N_{{ \\text{{obs}} }}$']
                          == my_obs_windows]
this_df = this_df[this_df['$\\epsilon_{{\\text{{Target}}}}$'] == my_epsilon]
this_df = this_df[this_df['$\\Delta_{{\\text{{Target}}}}$'] == my_bm_cvar_tgt] 
this_df_agg = pd.concat(
    [this_df_agg, df_bm_agg[df_bm_agg.Solver.str.contains("GM")]])
this_df = pd.concat(
    [this_df, df_bm[df_bm.Solver.str.contains("GM")]])
this_df_agg["Relative Efficiency"] = (
    1 - this_df_agg["Truth"]) * this_df_agg["Truth"]
this_df_agg["Relative Efficiency"] = this_df_agg["Relative Efficiency"] / \
    (this_df_agg["Cost Mean"] * this_df_agg["MSE"])

this_df_agg.loc[this_df_agg.Solver.str.startswith("CBREE"), "Solver"] = "CBREE" 
this_df.loc[this_df.Solver.str.startswith("CBREE"), "Solver"] = "CBREE" 
fig = ree.make_efficiency_plot(this_df_agg,
                               this_df,
                               "Sample Size",
                           "Relative Efficiency",
                           "Estimate",
                           facet_row="Solver",
                           shared_secondary_y_axes=True)
fig.show()
fig_title = "eficiency plot " + prob + " solver vs sample size"
fig.write_image(f"{fig_title}.pdf".replace(" ", "_").lower())
fig_description = f"Solving the {prob} with the CBREE method and two benchmark methods (row). \
We vary the sample size ($x$-axis) and show for each sample size two quantities. \
There is an estimate of the relative efficiency (left $y$-axis) and \
a boxplot of the corresponding ${int(2*this_df_agg.Seed.unique()[0]+1)}$ empirical estimates of the failure probability (righ $y$-axis). \
The other parameters of the CBREE method are $\\Delta_{{\\text{{Target}}}} = {my_bm_cvar_tgt}$, \
$N_\\text{{obs}} = {my_obs_windows}$ and \
$\\epsilon_{{\\text{{Target}}}} = {my_epsilon}$."
with open(f"{fig_title} desc.tex".replace(" ", "_").lower(), "w") as file:
    file.write(fig_description)
display(Markdown(fig_description))

Solving the Nonlinear Oscillator Problem with the CBREE method and two benchmark methods (row). We vary the sample size ($x$-axis) and show for each sample size two quantities. There is an estimate of the relative efficiency (left $y$-axis) and a boxplot of the corresponding $100$ empirical estimates of the failure probability (righ $y$-axis). The other parameters of the CBREE method are $\Delta_{\text{Target}} = 1$, $N_\text{obs} = 2$ and $\epsilon_{\text{Target}} = 1$.

## Parameter impact on rel. Efficiency

In [27]:
# filter
my_mixture_model = "GM"
my_obs_windows = [0,2,5]
my_epsilon = 1
my_bm_cvar_tgt = [1,2,5]
my_sample_size=6000
this_df_agg = df_agg.query("Problem == @prob & `Averaging Method`=='Average Estimate' & mixture_model==@my_mixture_model & `Sample Size` == @my_sample_size")
this_df_agg = this_df_agg[this_df_agg['$N_{{ \\text{{obs}} }}$'].isin(my_obs_windows)]
this_df_agg = this_df_agg[this_df_agg['$\\epsilon_{{\\text{{Target}}}}$'] == my_epsilon]
this_df = df.query("Problem == @prob & `Averaging Method`=='Average Estimate' & mixture_model==@my_mixture_model & `Sample Size` == @my_sample_size")
this_df = this_df[this_df['$N_{{ \\text{{obs}} }}$'].isin(my_obs_windows)]
this_df = this_df[this_df['$\\epsilon_{{\\text{{Target}}}}$'] == my_epsilon]
this_df_agg.loc[:,"Relative Efficiency"] = (
    1 - this_df_agg["Truth"]) * this_df_agg["Truth"]
this_df_agg.loc[:,"Relative Efficiency"] = this_df_agg["Relative Efficiency"] / \
    (this_df_agg["Cost Mean"] * this_df_agg["MSE"])

# nice columns names
columns = ['$\\Delta_{{\\text{{Target}}}}$']
# new_names = {k:LATEX_TO_HTML[v] for v in columns} 
for dat in [this_df, this_df_agg]:
    for c in columns:
        dat.loc[:,c] = dat[c].astype("int").astype("string")
        
fig = ree.make_efficiency_plot(this_df_agg,
                               this_df,
'$\\Delta_{{\\text{{Target}}}}$',
                           "Relative Efficiency",
                           "Estimate",
                           facet_row='$N_{{ \\text{{obs}} }}$',
                           facet_row_prefix = LATEX_TO_HTML['$N_{{ \\text{{obs}} }}$'],
                           shared_secondary_y_axes=True,
                           facet_name_sep = ' = ',
                           x_axis_sorting_key=lambda x: x.astype("int"),
                           labels = LATEX_TO_HTML)
# overwrite N_obs = 0
old_a = LATEX_TO_HTML[DF_COLUMNS_TO_LATEX["observation_window"]] + " = 0"
new_a = "No Divergence Check"
fig.for_each_annotation(
    lambda a: a.update(text = new_a if a.text.startswith(old_a) else a.text))
fig.show()
fig_title = "eficiency plot " + prob + " delta_tgt vs obs_window"
fig.write_image(f"{fig_title}.pdf".replace(" ", "_").lower())
fig_description = f"Solving the {prob} with the CBREE method with ${my_sample_size}$ samples. \
We vary the parameter $\\Delta_{{\\text{{Target}}}}$ ($x$-axis) and \
the length of the observation window $N_\\text{{obs}}$ (row). \
For each parameter choice we plot an estimate of the relative efficiency (left $y$-axis) and \
a boxplot of the corresponding ${int(2*this_df_agg.Seed.unique()[0]+1)}$ empirical estimates of the failure probability (righ $y$-axis). \
The parameter $\\epsilon_{{\\text{{Target}}}} = {my_epsilon}$ is fixed."
with open(f"{fig_title} desc.tex".replace(" ", "_").lower(), "w") as file:
    file.write(fig_description)
display(Markdown(fig_description))

Solving the Nonlinear Oscillator Problem with the CBREE method with $6000$ samples. We vary the parameter $\Delta_{\text{Target}}$ ($x$-axis) and the length of the observation window $N_\text{obs}$ (row). For each parameter choice we plot an estimate of the relative efficiency (left $y$-axis) and a boxplot of the corresponding $100$ empirical estimates of the failure probability (righ $y$-axis). The parameter $\epsilon_{\text{Target}} = 1$ is fixed.